In [1]:
from datasets import load_dataset


# Dataset id from huggingface.co/dataset
dataset_id = "dair-ai/emotion"

# Load raw dataset
raw_dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")

# Train dataset size: 16000
# Test dataset size: 2000

Train dataset size: 16000
Test dataset size: 2000


In [2]:
from random import randrange


random_id = randrange(len(raw_dataset["train"]))
raw_dataset["train"][random_id]
# {'text': 'i also like to listen to jazz whilst painting it makes me feel more artistic and ambitious actually look to the rainbow', 'label': 1}

{'text': 'i start enjoying it and it becomes part of the general fabric of feeling joyful about your everyday activities about just being',
 'label': 1}

In [4]:
!wget https://raw.githubusercontent.com/huggingface/optimum-neuron/main/notebooks/text-classification/scripts/train.py

--2025-11-18 19:12:27--  https://raw.githubusercontent.com/huggingface/optimum-neuron/main/notebooks/text-classification/scripts/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4548 (4.4K) [text/plain]
Saving to: ‘train.py’

train.py            100%[===================>]   4.44K  --.-KB/s    in 0s      

2025-11-18 19:12:27 (66.9 MB/s) - ‘train.py’ saved [4548/4548]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
%env NEURON_RT_INSPECT_ENABLE=0
%env OMP_NUM_THREADS=8

In [ ]:
%env NEURON_RT_STOCHASTIC_ROUNDING_EN=1

In [7]:
!torchrun --nproc_per_node=2 train.py \
 --model_id bert-base-uncased \
 --lr 5e-5 \
 --per_device_train_batch_size 8 \
 --bf16 True \
 --epochs 3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


W1118 19:41:08.726000 6749 torch/distributed/run.py:774] 
W1118 19:41:08.726000 6749 torch/distributed/run.py:774] *****************************************
W1118 19:41:08.726000 6749 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1118 19:41:08.726000 6749 torch/distributed/run.py:774] *****************************************
Map: 100%|████████████████████████| 2000/2000 [00:00<00:00, 12529.19 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-11-18 19:41:15.482259: W neuron/pjrt-api/neuronpjrt.cc:1972] Use PJRT C-API 0.73 as client did not

In [ ]:
!python eval.py --model_path ./bert-base-uncased-finetuned

In [11]:
%env NEURON_RT_INSPECT_ENABLE=1
%env NEURON_RT_INSPECT_SYSTEM_PROFILE=1
%env NEURON_RT_INSPECT_DEVICE_PROFILE=1
%env NEURON_RT_INSPECT_OUTPUT_DIR=./bert_profile_device

env: NEURON_RT_INSPECT_ENABLE=1
env: NEURON_RT_INSPECT_SYSTEM_PROFILE=1
env: NEURON_RT_INSPECT_DEVICE_PROFILE=1
env: NEURON_RT_INSPECT_OUTPUT_DIR=./bert_profile_device


In [12]:
!torchrun --nproc_per_node=2 train.py \
    --model_id bert-base-uncased \
    --lr 5e-5 \
    --per_device_train_batch_size 8 \
    --bf16 True \
    --epochs 3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


W1118 20:35:25.227000 12256 torch/distributed/run.py:774] 
W1118 20:35:25.227000 12256 torch/distributed/run.py:774] *****************************************
W1118 20:35:25.227000 12256 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1118 20:35:25.227000 12256 torch/distributed/run.py:774] *****************************************
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-11-18 20:35:31.781305: W neuron/pjrt-api/neuronpjrt.cc:1972] Use PJRT C-API 0.73 as client did not specify a PJRT C-API version
Some weights of BertForSequenceClassification w

In [10]:
!neuron-profile view -d ./bert_profile --output-format perfetto

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
